In [2]:
from src.training.utils import load_checkpoint_for_inference, load_config, create_model
from src.data.tokenization import detokenize
from src.data.dataset_util import remove_delay_interleaving
import torch
import torchaudio
from typing import Tuple
from pathlib import Path

device = torch.device("cpu")
config = load_config("../config/model/experiment.yaml")
model = create_model(config).to(device)
load_checkpoint_for_inference(model, "../models/experiment/checkpoints/checkpoint_epoch_450.pt", device)

test_ds = torch.load("../data/experiment/datasets/train.pt", weights_only=False)
len(test_ds)

Loading checkpoint from ../models/experiment/checkpoints/checkpoint_epoch_450.pt
Checkpoint info:
  Epoch: 449
  Step: 1800
  Metrics: {'train_loss': 0.06740963631974799, 'train_accuracy': 0.9766969233751297, 'val_loss': 13.881664085388184, 'val_accuracy': 0.23289999961853028}


111

In [4]:
SAMPLE_NUMBER = 4

def get_generated_audio(
    model: torch.nn.Module,
    example: Tuple[torch.Tensor, torch.Tensor],
    save_dir: str = "./outputs/",
    greedy: bool = True,
    **sampling_args,
):
    src, tgt = example
    src = src.to(device).unsqueeze(0)
    tgt = tgt.to(device)
    save_dir = Path(save_dir)
    start_tokens = torch.tensor(
        [[2048 for _ in range(4)]],
        dtype=torch.int,
        device=device,
    )
    if greedy:
        out = model.generate_greedy(
            src,
            max_len=1505,
            start_tokens=start_tokens,
        )
    else:
        out = model.generate(
            src,
            max_len=1505,
            start_tokens=start_tokens,
            **sampling_args,
        )
    num_nums = 1
    for i in out.shape:
        num_nums *= i
    print(f"{num_nums} entries")
    print(f"num entries correct: {(out[0] == tgt).sum().item()}")
    print(out)
    out = detokenize(remove_delay_interleaving(out[0]))
    torchaudio.save(
        save_dir / "model_gen.wav",
        out,
        32000,
    )
    torchaudio.save(
        save_dir / "src.wav",
        detokenize(remove_delay_interleaving(src[0])),
        32000,
    )
    torchaudio.save(
        save_dir / "tgt.wav",
        detokenize(remove_delay_interleaving(tgt)),
        32000,
    )
    
get_generated_audio(model, test_ds[SAMPLE_NUMBER])

6020 entries
num entries correct: 1241
tensor([[[2048,  166,  166,  ...,   83,   83,   83],
         [2048, 2048, 1931,  ..., 2044, 2044, 2044],
         [2048, 2048, 2048,  ..., 2019, 2019, 2019],
         [2048, 2048, 2048,  ..., 1770, 1770, 1770]]])


In [8]:
import torch
import torch.nn.functional as F
from pathlib import Path
from typing import Dict, Tuple
from tqdm import tqdm
import yaml
from src.model.transformer import EncoderDecoderTransformer

# Assume these utility functions are imported from your main training script's context
# from .utils import load_config, create_model, setup_device, load_checkpoint_for_training

# --- Placeholder/Mock Classes for Execution (Replace with your actual imports) ---
# NOTE: For the script to run, you need the real imports for your model, utilities, and a mock dataset.

# Mock DataLoader and Dataset for easy testing on a single batch
class MockDataset(torch.utils.data.Dataset):
    def __init__(self, config):
        # Create mock data based on config
        self.num_codebooks = config["model"]["num_codebooks"] # 4
        self.seq_len = config["model"]["max_seq_len"] # 1505
        self.padding_idx = config["model"]["padding_idx"] # 2048
        
        # A simple, deterministic sequence for easy checking
        # Start with all padding (2048) at t=0
        src = torch.full((self.num_codebooks, self.seq_len), self.padding_idx, dtype=torch.long)
        tgt = torch.full((self.num_codebooks, self.seq_len), self.padding_idx, dtype=torch.long)
        
        # Fill in codebook tokens with unique, non-padding values (e.g., 100, 200, 300...)
        # CB_idx: 0, 1, 2, 3
        # Start Time: 1, 2, 3, 4 (positions 1, 2, 3, 4)
        for cb_idx in range(self.num_codebooks):
            # Non-padding tokens start at position cb_idx + 1
            # Token values: 100+cb_idx * 10 (e.g., 100, 110, 120, 130)
            token_value = 100 + cb_idx * 10 
            
            # Fill tokens from position (cb_idx + 1) to end
            src[cb_idx, cb_idx + 1:] = torch.arange(
                token_value, token_value + self.seq_len - (cb_idx + 1), dtype=torch.long
            ) % (self.padding_idx - 1) # Modulo to keep in range [0, 2047]
            tgt[cb_idx, cb_idx + 1:] = src[cb_idx, cb_idx + 1:] # Target is same as source for this test

        # Set the first position (t=0) to a sentinel non-padding token if needed, 
        # but based on your description, t=0 is padding for all codebooks.
        # We assume the model expects to predict the sequence from t=1 onwards.
        # Let's align the data with a standard [BOS] token (which is often index 0) if padding_idx=2048 is only used for padding.
        
        # Based on your description: "the first position is always 2048 for all codebooks"
        # and "the first codebook has its first actual token at the second position."
        # This means the target sequence TGT has a start token at T=0 (position 0) and the real sequence starts at T=1.
        self.data = [(src.unsqueeze(0), tgt.unsqueeze(0))] # Single batch of size 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# --- Debug Function ---

def debug_generation_caching(config: Dict, model_class: torch.nn.Module):
    """
    Simulates step-by-step teacher-forced decoding with caching
    to debug the core decode/cache mechanism.
    """
    print("--- Starting Generation Caching Debug Script ---")
    
    # Setup device
    device = torch.device("cpu")
    print(f"Device: {device}")

    # 1. Instantiate Model
    print("1. Creating and Loading Model...")
    # NOTE: You must provide the actual 'create_model' and 'load_checkpoint_for_training'
    # We mock them here for concept.
    try:
        # Replace with your actual model instantiation
        model = model_class(**config["model"]).to(device)
    except Exception as e:
        print(f"Error creating model: {e}")
        return

    # Load a checkpoint to ensure weights are correct
    checkpoint_path = Path(config["training"]["resume_from_checkpoint"])
    try:
        # Load weights only for this debug script.
        # This requires the actual structure of your checkpoint file.
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint["model_state_dict"])
        print(f"Model weights loaded from {checkpoint_path}")
    except Exception as e:
        print(f"Warning: Could not load checkpoint. Running with random weights. Error: {e}")
        print("This test is only valid if you load a trained checkpoint.")
    
    model.eval()

    # 2. Prepare Data (A single example from the training set)
    # We use a mock dataset here, but ideally, you'd load one batch from your real train_loader
    print("2. Preparing Test Data...")
    mock_dataset = MockDataset(config)
    
    # Get the single batch: src, tgt [B=1, C=4, T=1505]
    src, tgt = mock_dataset[0] 
    src = src.to(device)
    tgt = tgt.to(device)
    batch_size, num_codebooks, seq_len = tgt.shape
    
    print(f"Data shape: (B={batch_size}, C={num_codebooks}, T={seq_len})")
    
    # Initial sequence length to start decoding: 1 (the start token at T=0)
    # We will simulate prediction from T=1 up to T=seq_len-1 (1504 total steps)
    start_t = 0
    end_t = seq_len - 1

    # 3. Encode Source (Encoder part of generation)
    memory = model.encode(src)

    # 4. Step-by-Step Teacher-Forced Decoding
    print("\n3. Starting Step-by-Step Decoding Simulation...")
    
    # tgt_so_far holds the predicted sequence (initially just the first token)
    # Start with the first token of the target sequence (t=0)
    tgt_so_far = tgt[:, :, start_t].unsqueeze(-1) # [1, 4, 1]
    cache = [
        {"self_attn": None, "cross_attn": None}
        for _ in range(model.num_decoder_layers)
    ]
    
    total_correct = 0
    total_non_padding = 0

    # Loop from t=1 to T-1
    for t in tqdm(range(start_t, end_t)):
        # Input to decoder is the token at 't' (the last token in tgt_so_far)
        # This is the "teacher-forced" input.
        current_tokens = tgt_so_far[:, :, -1:] 
        
        # Expected correct tokens to be predicted at position t+1
        expected_next_tokens = tgt[:, :, t + 1] # [1, 4]
        
        # Decode one step with caching
        logits, cache = model.decode(
            current_tokens,
            memory,
            cache=cache,
            use_cache=True,
            is_causal=True,
        ) # logits: [1, 4, 1, V_proj]

        # Get greedy prediction: [1, 4]
        # NOTE: argmax is over V_proj (size 2048, indices 0-2047)
        predicted_next_tokens = logits[:, :, -1, :].argmax(dim=-1) 

        # *** CRITICAL CHECK 1: The Padding/Delay Logic Mismatch ***
        # The correct target indices are 0-2048. Predicted indices are 0-2047.
        # This is the expected mismatch due to the model's output projection size.
        # We must align the prediction to the target's vocab space [0, 2048] for comparison.
        # Since 2048 is the padding index, we ONLY care about 0-2047.
        
        # Check all codebooks
        for cb_idx in range(num_codebooks):
            expected = expected_next_tokens[0, cb_idx].item()
            predicted = predicted_next_tokens[0, cb_idx].item()
            
            # Non-padding token indices are [0, 2047]
            if expected != config["model"]["padding_idx"]:
                total_non_padding += 1
                if expected == predicted:
                    total_correct += 1
                else:
                    # Log the first few errors for inspection
                    if total_non_padding < 100 or total_non_padding % 500 == 0:
                         print(
                             f"\n🚨 Error at T={t+1} (Position {t+2}), CB={cb_idx}: "
                             f"Expected={expected}, Predicted={predicted}. "
                             f"Previous Token={current_tokens[0, cb_idx, 0].item()}"
                         )

            # Manually force the prediction to be correct for the next step 
            # (Teacher forcing step)
            # This is crucial: we must ensure the *correct* token is fed back.
            predicted_next_tokens[0, cb_idx] = expected_next_tokens[0, cb_idx]

        # Append the CORRECT token to the sequence for the next step
        tgt_so_far = torch.cat([tgt_so_far, predicted_next_tokens.unsqueeze(-1)], dim=-1)

    # 5. Final Report
    simulation_accuracy = total_correct / total_non_padding if total_non_padding > 0 else 0.0
    
    print("\n" + "=" * 50)
    print("         DECODING CACHING SIMULATION REPORT")
    print("=" * 50)
    print(f"Total Non-Padding Tokens Checked: {total_non_padding}")
    print(f"Total Correct Predictions (Teacher-Forced Cache): {total_correct}")
    print(f"Simulation Accuracy: {simulation_accuracy:.4f} (Expected: ~0.9400)")
    print("=" * 50)
    
    if simulation_accuracy < 0.90:
        print("\n**Conclusion:** The model's **caching and decode logic is fundamentally flawed**.")
        print("This confirms the internal decode logic with incremental steps (KV cache) is broken,")
        print("even when given the correct previous token (teacher forcing).")
        
    elif simulation_accuracy > 0.93:
        print("\n**Conclusion:** The model's **caching and decode logic works correctly**.")
        print("The 0% generation accuracy is due to the **argmax/sampling** or the **delay-forcing logic**")
        print("within your `generate()` function, causing an immediate catastrophic error cascade.")
        
        # *** Next Debug Step: The Delay Logic in generate() ***
        print("\n--- NEXT DEBUG STEP (Focus on generate() Logic) ---")
        print("The most suspicious part of your `generate()` is the delay condition:")
        print("`if current_position <= cb_idx:`")
        print("If `tgt` starts at length 1 (the start token), then at the first loop step (t=0),")
        print("`current_position` is 1. CB0 (cb_idx=0) fails the condition (1 <= 0 is False) and predicts.")
        print("CB1 (cb_idx=1) passes the condition (1 <= 1 is True) and forces 2048.")
        print("This seems correct for a sequence starting at T=0, but verify against your data preparation.")
        print("Try changing `current_position <= cb_idx` to `current_position < cb_idx + 1` or similar.")


# --- Execution Block (You need to run this with your actual environment) ---

# Mock the utilities to load the config
def load_config(config_path):
    # Use the YAML data provided in the prompt
    config_yaml = """
data:
  train_path: "data/main/datasets/train.pt"
  val_path: "data/main/datasets/val.pt"

# Model architecture
model:
  vocab_size: 2049
  num_codebooks: 4
  d_model: 128
  nhead: 8
  num_encoder_layers: 8
  num_decoder_layers: 8
  dim_feedforward: 512
  dropout: 0.1
  activation: "relu"         
  norm_first: false          
  max_seq_len: 1505
  padding_idx: 2048
  scale_embeddings: true


training:
  num_epochs: 300
  # Effective batch size = batch_size * gradient_accumulation_steps
  batch_size: 8
  gradient_accumulation_steps: 4
  
  # Learning rate scheduler
  scheduler:
    type: "cosine"           # Options: cosine, linear, step, plateau
    warmup_steps: 1000       # Warmup for 1000 steps
    total_steps: 50000       # Total training steps (auto-calculated if omitted)
    min_lr: 1.0e-6           # Minimum learning rate
    
    # For 'step' scheduler:
    # step_size: 10          # Decay every N epochs
    # gamma: 0.5             # Multiply LR by this factor
    
    # For 'plateau' scheduler:
    # patience: 3            # Wait N epochs before reducing
    # factor: 0.5            # Multiply LR by this factor

  # Optimizer
  optimizer:
    type: "adamw"
    lr: 0.0001 
    betas: [0.9, 0.98]
    eps: 1.0e-9
    weight_decay: 0.01
  
  # Logging and checkpointing
  output_dir: "models/small"
  log_interval: 2           # Log metrics every N steps
  save_interval: 50      # Save checkpoint every N epochs
  num_workers: 2             # DataLoader workers
  
  # Resume training (optional)
  resume_from_checkpoint: "../models/small/checkpoints/checkpoint_epoch_200.pt" # Assuming you've trained to 200
"""
    return yaml.safe_load(config_yaml)

# Get the config
config_data = load_config(None)

# Execute the debug function with your actual EncoderDecoderTransformer class
debug_generation_caching(config_data, EncoderDecoderTransformer)

--- Starting Generation Caching Debug Script ---
Device: cpu
1. Creating and Loading Model...
Model weights loaded from ../models/small/checkpoints/checkpoint_epoch_200.pt
2. Preparing Test Data...
Data shape: (B=1, C=4, T=1505)

3. Starting Step-by-Step Decoding Simulation...


  1%|▌                                                           | 13/1504 [00:00<00:12, 119.12it/s]


🚨 Error at T=1 (Position 2), CB=0: Expected=100, Predicted=1673. Previous Token=2048

🚨 Error at T=2 (Position 3), CB=0: Expected=101, Predicted=100. Previous Token=100

🚨 Error at T=2 (Position 3), CB=1: Expected=110, Predicted=1398. Previous Token=2048

🚨 Error at T=3 (Position 4), CB=0: Expected=102, Predicted=641. Previous Token=101

🚨 Error at T=3 (Position 4), CB=1: Expected=111, Predicted=1115. Previous Token=110

🚨 Error at T=3 (Position 4), CB=2: Expected=120, Predicted=754. Previous Token=2048

🚨 Error at T=4 (Position 5), CB=0: Expected=103, Predicted=786. Previous Token=102

🚨 Error at T=4 (Position 5), CB=1: Expected=112, Predicted=940. Previous Token=111

🚨 Error at T=4 (Position 5), CB=2: Expected=121, Predicted=120. Previous Token=120

🚨 Error at T=4 (Position 5), CB=3: Expected=130, Predicted=642. Previous Token=2048

🚨 Error at T=5 (Position 6), CB=0: Expected=104, Predicted=103. Previous Token=103

🚨 Error at T=5 (Position 6), CB=1: Expected=113, Predicted=1646. Pre

 11%|██████▎                                                    | 161/1504 [00:00<00:07, 178.38it/s]


🚨 Error at T=127 (Position 128), CB=1: Expected=235, Predicted=452. Previous Token=234


...


🚨 Error at T=1252 (Position 1253), CB=1: Expected=1360, Predicted=1359. Previous Token=1359


 93%|█████████████████████████████████████████████████████▊    | 1396/1504 [00:10<00:01, 104.33it/s]


🚨 Error at T=1377 (Position 1378), CB=1: Expected=1485, Predicted=1484. Previous Token=1484


100%|██████████████████████████████████████████████████████████| 1504/1504 [00:11<00:00, 133.30it/s]



🚨 Error at T=1502 (Position 1503), CB=1: Expected=1610, Predicted=1609. Previous Token=1609

         DECODING CACHING SIMULATION REPORT
Total Non-Padding Tokens Checked: 6010
Total Correct Predictions (Teacher-Forced Cache): 1
Simulation Accuracy: 0.0002 (Expected: ~0.9400)

**Conclusion:** The model's **caching and decode logic is fundamentally flawed**.
This confirms the internal decode logic with incremental steps (KV cache) is broken,
even when given the correct previous token (teacher forcing).
